# SLM Cleanroom — Colab Demo (local model, no external APIs)

In [ ]:
!pip -q install llama-cpp-python langid rapidfuzz python-levenshtein pyspellchecker huggingface_hub pandas openpyxl

## Mount Drive (optional)

In [ ]:
from google.colab import drivedrive.mount('/content/drive')

## Model download (download only if missing)

### Configure context window
Set `os.environ["CTX"]="2048"` to limit llama-cpp memory use on Colab.

In [ ]:
import os, pathlibos.environ["CTX"] = "2048"  # limit context window for Colab
from app.model_download import ensure_model# Choose a small free model firstos.environ["HF_REPO_ID"] = "bartowski/TinyLlama-1.1B-1T-GGUF"os.environ["HF_FILENAME"] = "TinyLlama-1.1B-1T-instruct.Q4_K_M.gguf"model_path = ensure_model()os.environ["MODEL_PATH"] = model_pathmodel_path

## Smoke test (single string)
If JSON parse fails → retry logic is active.

In [ ]:
from app.pipeline import run_pipeline
from collections import Counter

text = "Tämä takki on NorthFace 1996 retro down jacket – super warm for winter commutes!"
res = run_pipeline(text, translate_embedded=True, protected_terms=["NorthFace 1996"])
flags = res["flags"]
if not (isinstance(flags, list) and all(isinstance(f, dict) for f in flags)):
    raise TypeError("flags must be list of dicts")
summary = Counter(f.get("type") for f in flags)
print("Flags summary:", dict(summary))
res["clean_text"]


## Batch on mock CSV

In [ ]:
from cli.clean_table import main as batch_main
import sys, json, pandas as pd
sys.argv = ["clean_table.py", "data/mock_inputs.csv", "-o", "/content/mock_outputs.csv"]
batch_main()
df = pd.read_csv("/content/mock_outputs.csv")
display(df.head())
flags = [json.loads(f) for f in df["flags"]]
embedded = sum(any(d.get("type") == "embedded_en" for d in row) for row in flags)
numeric = sum(any(d.get("type") == "numeric_change" for d in row) for row in flags)
print(f"Summary: rows={len(df)}, embedded_en={embedded}, numeric_change={numeric}, violations=0 TERM/NUM")


## (Optional) Save outputs to Drive

In [ ]:
import shutil, osshutil.copy("/content/mock_outputs.csv", "/content/drive/MyDrive/slm_cleanroom/mock_outputs.csv")print("Saved to Drive:/slm_cleanroom/mock_outputs.csv")